In [1]:
import pandas as pd
from pyproj import Proj, transform
import fiona
from fiona.crs import from_epsg
import logging
import sys
from aabbtree import AABB, AABBTree


# outProj = Proj(init='epsg:3857')
# inProj = Proj(init='epsg:4326') # It is equal to WSG

In [53]:
# Parametros

espacamento = 50
n = 5 # multiplicador da area de amortização

In [3]:
# Carregar base em metros da area de preservação do chico mendes
chico_mendes = fiona.open("Mapas_sim/shapes_sim_meters/area_chico_mendes.shp")
print (chico_mendes.schema)

{'properties': OrderedDict([('id', 'float:19'), ('Unidade_de', 'str:254'), ('Categoria', 'str:254'), ('Ano_de_Cri', 'int:9'), ('Zoneamento', 'str:254'), ('Área_hecta', 'float:33.15'), ('Jurisdição', 'str:254'), ('Plano_de_M', 'str:254')]), 'geometry': 'Polygon'}


In [4]:
# Estrutura dos arquivos com dados
chico_mendes[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('id', 897.0),
              ('Unidade_de', 'PARQUE NATURAL MUNICIPAL CHICO MENDES'),
              ('Categoria', 'PROTEÇÃO INTEGRAL'),
              ('Ano_de_Cri', None),
              ('Zoneamento', None),
              ('Área_hecta', 40.65),
              ('Jurisdição', 'MUNICIPAL'),
              ('Plano_de_M', None)]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-4838947.215019874, -2634641.6353100897),
    (-4838947.215019874, -2634641.6353100897),
    (-4838911.530555228, -2634641.478797528),
    (-4838880.228516291, -2634641.665790127),
    (-4838865.3888489315, -2634642.0394124887),
    (-4838850.586696242, -2634643.1607634327),
    (-4838835.858904972, -2634645.0269402442),
    (-4838821.242655828, -2634647.6333469907),
    (-4838806.774795566, -2634650.973331637),
    (-4838792.491503024, -2634655.038669846),
    (-4838778.428845712, -2634659.8189602373),
    (-4838764.622223226, -2634665.3021081625),
    (

In [5]:
listaCoordenadas = chico_mendes[0]['geometry']['coordinates']

In [6]:
type(listaCoordenadas)

list

In [7]:
# retirar ponto x máximo e ponto x mínimo, referente a projeção 3857
# analogo para o ymin e ymax

xmin = 999999999999999
xmax = -99999999999999
ymax = xmax
ymin = xmin
for elem in listaCoordenadas[0]:
    
    if elem[0] > xmax:
        xmax = elem[0]
    elif elem[0] < xmin:
        xmin = elem[0]
        
    if elem[1] > ymax:
        ymax = elem[1]
    elif elem[1] < ymin:
        ymin = elem[1]

In [8]:
print('X Max: ', xmax, '\nX Min: ', xmin)
print('YMax: ', ymax, '\nY Min: ', ymin)

X Max:  -4838636.088731658 
X Min:  -4839587.213704263
YMax:  -2634641.478797528 
Y Min:  -2635360.089792147


In [54]:
yminAmortizado = ymin + (-2*espacamento) 
ymaxAmortizado = ymax + (2*espacamento) 
xminAmortizado = xmin + (-2*espacamento) 
xmaxAmortizado = xmax + (2*espacamento)

In [55]:
print('X Max: ', xmaxAmortizado, '\nX Min: ', xminAmortizado)
print('YMax: ', ymaxAmortizado, '\nY Min: ', yminAmortizado)

X Max:  -4838536.088731658 
X Min:  -4839687.213704263
YMax:  -2634541.478797528 
Y Min:  -2635460.089792147


In [56]:
tree = AABBTree()

treeEstudo = AABBTree()
# (xmin, xmax), (ymin, ymax)
area_estudo = AABB([(xmin, xmax), (ymin, ymax)])

area_amortizada = AABB([(xminAmortizado, xmaxAmortizado), (yminAmortizado, ymaxAmortizado)])


In [57]:
# aabb2 = AABB([(10,15), (10,15)])

tree.add(area_amortizada, 'envol')
treeEstudo.add(area_estudo)
a = tree.does_overlap(area_estudo)

print(a)


True


In [14]:
escolas = fiona.open("Mapas_sim/shapes_sim_meters/escola_metros.shp")
print (escolas.schema)

{'properties': OrderedDict([('INEP', 'int:9'), ('Escola', 'str:254'), ('Rede', 'str:254'), ('Localizaçã', 'str:254'), ('Município', 'str:254'), ('Bairro', 'str:254'), ('Endereço', 'str:254'), ('Telefone', 'str:254'), ('Total_Matr', 'float:19'), ('Infantil', 'float:19'), ('Creche', 'float:19'), ('PreEscola', 'float:19'), ('Fundamenta', 'float:19'), ('Fundament0', 'float:19'), ('Fundament1', 'float:19'), ('Ensino_Med', 'float:19'), ('Profission', 'int:9'), ('EJA', 'int:9'), ('Especial', 'int:9'), ('Latitude', 'float:33.15'), ('Longitude', 'float:33.15')]), 'geometry': 'Point'}


In [22]:
escolas[0]['geometry']

{'type': 'Point', 'coordinates': (-4809884.843755053, -2627191.2669907277)}

In [24]:
a,b = escolas[0]['geometry']['coordinates']


In [26]:
a

-4809884.843755053

In [34]:
listaEscolas = []

for escola in escolas:
    x, y = escola['geometry']['coordinates']
    
    t = AABB([(x, x), (y, y)])
    
    if(tree.does_overlap(t)):
        listaEscolas.append(escola)
    

In [37]:
sink_schema = escolas.schema.copy()
with fiona.open(
        'escola_estudo.shp', 'w',
        crs=from_epsg(3857),
        driver=escolas.driver,
        schema=sink_schema,
        ) as sink:
    
    for e in listaEscolas:
        sink.write(e)

In [38]:
saude = fiona.open("Mapas_sim/shapes_sim_meters/saude_metros.shp")
print (saude.schema)

{'properties': OrderedDict([('Município', 'str:254'), ('Código_CNE', 'str:254'), ('Razão_Soci', 'str:254'), ('Estabeleci', 'str:254'), ('CNPJ_Mante', 'str:254'), ('Endereço', 'str:254'), ('Telefone', 'str:254'), ('Email_ou_F', 'str:254'), ('Tipo', 'str:254'), ('Gestão', 'str:254')]), 'geometry': 'Point'}


In [39]:
listaSaude = []

for unidade in saude:
    x, y = unidade['geometry']['coordinates']
    
    t = AABB([(x, x), (y, y)])
    
    if(tree.does_overlap(t)):
        listaSaude.append(unidade)
    

In [40]:
listaSaude

[]

In [42]:
onibus = fiona.open("Mapas_sim/shapes_sim_meters/ponto_onibus_metros.shp")
print (onibus.schema)

{'properties': OrderedDict([('Nome_da_Li', 'str:254'), ('Número_da_', 'str:254'), ('Rota', 'str:254')]), 'geometry': 'MultiPoint'}


In [62]:
len(onibus)#['geometry']['coordinates'][176]


499

In [79]:
listaOnibus = []
i = 0
for linha in onibus:
    for ponto in linha['geometry']['coordinates']:
#         print(ponto)
        x, y = ponto
        
#         print(x, y)

        t = AABB([(x, x), (y, y)])

        if(tree.does_overlap(t)):
            listaOnibus.append(linha)
            print('oi')
            break

# print(i)

In [77]:
listaOnibus

[]